In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image dimensions
img_height = 128  # Example height, change as needed
img_width = 128   # Example width, change as needed

# Create a data generator with rescaling and augmentation
datagen = ImageDataGenerator(
    rescale=1./255,             # Rescale pixel values
    rotation_range=40,          # Random rotation between -40 to 40 degrees
    zoom_range=0.2,             # Random zoom
    horizontal_flip=True,       # Random horizontal flips
    vertical_flip=True          # Random vertical flips (optional)
)
# Fit generator to your dataset
train_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/images.cv_jzk6llhf18tm3k0kyttxz/dataSET/train',
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical'  # or 'binary' depending on your task
)

In [ ]:
from tensorflow.keras.applications import VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

In [ ]:
base_model.trainable = False

In [ ]:
from tensorflow.keras import layers, models

x = base_model.output
x = layers.GlobalAveragePooling2D()(x) # Better than Flatten for pre-trained models
x = layers.Dense(1024, activation='relu')(x)
NUM_CLASSES = train_generator.num_classes # Define NUM_CLASSES from the training generator
predictions = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = models.Model(inputs=base_model.input, outputs=predictions)
print(NUM_CLASSES)


In [ ]:
# Unfreeze the last few layers of the base model
base_model.trainable = True # Set base_model to trainable first
N_LAYERS_TO_UNFREEZE = 4 # Define the number of layers to unfreeze. Change this value as needed.
for layer in base_model.layers[:-N_LAYERS_TO_UNFREEZE]:
    layer.trainable = False

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16, ResNet50, MobileNet, InceptionV3, EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
import os
import pandas as pd

# --- 1. CONFIGURATION ---
# >>> IMPORTANT: Replace these with your actual values <<<
# Replace 'path/to/your/fish_dataset' with the actual path to the root of your dataset
DATASET_PATH = 'path/to/your/fish_dataset'
IMAGE_SIZE = 224      # Standard size for VGG, ResNet, MobileNet
BATCH_SIZE = 32
EPOCHS_SCRATCH = 25   # More epochs for scratch model
EPOCHS_TRANSFER = 10  # Fewer epochs for transfer learning steps
BEST_MODEL_FILEPATH = 'best_fish_classifier_model.h5' # The final saved model name

# --- 2. DATA GENERATORS ---

# Use augmentation for training data to improve generalization
train_datagen = ImageDataGenerator(
    rescale=1./255, # Normalize
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Only normalization for validation data
validation_datagen = ImageDataGenerator(rescale=1./255)

try:
    train_generator = train_datagen.flow_from_directory(
        os.path.join(DATASET_PATH, 'train'),
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )
    validation_generator = validation_datagen.flow_from_directory(
        os.path.join(DATASET_PATH, 'validation'),
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )
except Exception as e:
    print(f"Error loading data: {e}. Please check DATASET_PATH and folder structure.")
    # Removed exit() to allow the cell to continue execution after printing the error
    # exit()

NUM_CLASSES = train_generator.num_classes

# --- 3. CALLBACKS for Saving the MAX Accuracy Model ---

# ModelCheckpoint saves the model with the highest 'val_accuracy'
checkpoint = ModelCheckpoint(
    filepath=BEST_MODEL_FILEPATH,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True, # This ensures only the highest accuracy model is saved
    verbose=1,
    # Removed save_format='h5' as it's not a valid argument
)

# EarlyStopping stops training if validation loss plateaus
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

results_list = []